<a href="https://colab.research.google.com/github/a-njan-a/Supply-chain-mapping/blob/main/Reuters_news_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests --upgrade --quiet
!pip install pandas --quiet
!pip install selenium --quiet
!pip install beautifulsoup4 --upgrade --quiet
!pip install webdriver-manager --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade selenium webdriver_manager

In [ ]:
import requests
import os
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

import time

from bs4 import BeautifulSoup

In [ ]:
selenium.__version__

'4.15.2'

In [ ]:
topics_url = 'https://www.reuters.com/tags/supply-chain/'

In [ ]:
response = requests.get(topics_url)

In [ ]:
response.status_code

In [ ]:
page_contents = response.text
page_contents[:1000]

In [ ]:
with open('webpage.html', 'w', encoding='utf-8' ) as f:
    f.write (page_contents)

In [ ]:
doc = BeautifulSoup(page_contents, 'html.parser')

In [ ]:
#Using selenium to scrape data

# Initialize the Chrome WebDriver (or choose a different browser)
# options = Options()
# options.add_experimental_option("detach", True)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
#                           options=options)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the URL
url = "https://www.reuters.com/tags/supply-chain/"  # Replace with the actual URL
driver.get(url)

all_headlines = []

while True:
    try:
        # Find and scrape the news headlines
        headline_tags = driver.find_elements(By.XPATH, "//a[@class='text__text__1FZLe text__dark-grey__3Ml43 text__inherit-font__1Y8w3 text__inherit-size__1DZJi link__underline_on_hover__2zGL4 link__inherit-line-height__2qjXx media-story-card__heading__eqhp9']")  # Adjust the XPath as needed

        for headline in headline_tags:
            all_headlines.append(headline.text)

        # Click the "Next" button (adjust the XPath as needed)
        next_button = driver.find_element(By.XPATH, "//button[@class='button__button__2Ecqi button__secondary__18moI button__round__1nYLA button__has-tooltip__1B4u0 search-results__page__3kjXX']")
        next_button.click()

        # Wait briefly to allow the next page to load
        time.sleep(2)  # Adjust the sleep duration as needed
    except Exception as e:
        # Handle exceptions, e.g., if there's no "Next" button or an error occurs
        print("An error occurred:", str(e))
        break

# Close the browser when done
driver.quit()

# Now, all_headlines contains headlines from all pages


In [ ]:
selection_class="text__text__1FZLe text__dark-grey__3Ml43 text__inherit-font__1Y8w3 text__inherit-size__1DZJi link__underline_on_hover__2zGL4 link__inherit-line-height__2qjXx media-story-card__heading__eqhp9"
headline_tags = doc.find_all('a', {'class' : selection_class})

In [ ]:
headline_tags[0]

In [ ]:
headlines = []

for tag in headline_tags:
    headlines.append(tag.text)

len(headlines)

In [ ]:
date_time = []

time_class = "text__text__1FZLe text__inherit-color__3208F text__regular__2N1Xr text__extra_small__1Mw6v body__base__22dCE body__extra_small_body__3QTYe media-story-card__time__2i9EK"
time_tags = doc.find_all('time', {'class' : time_class})

for tag in time_tags:
    date_time.append(tag['datetime'])

print(date_time)

In [ ]:
news_urls = []
base_url = "https://www.reuters.com"

for tag in headline_tags:
    news_urls.append(base_url + tag['href'])

print(news_urls)

Getting article

In [ ]:
news_page_url = news_urls[0]
news_page_url

In [ ]:
response = requests.get(news_page_url)

In [ ]:
news_doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
para_selection_class = "text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__small__1kGq2 body__full_width__ekUdw body__small_body__2vQyf article-body__paragraph__2-BtD"
news_tags = news_doc.find_all('p', { 'class' : para_selection_class})

In [ ]:
news_tags

In [ ]:
article_list = []

for tag in news_tags:
    article_list.append(tag.text)

article = " ".join([str(item) for item in article_list])
print(article)

In [ ]:
articles = []

for url in news_urls:
  response = requests.get(url)
  news_doc = BeautifulSoup(response.text, 'html.parser')
  para_selection_class = "text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__small__1kGq2 body__full_width__ekUdw body__small_body__2vQyf article-body__paragraph__2-BtD"

  news_tags = news_doc.find_all('p', { 'class' : para_selection_class})
  article_list = []

  for tag in news_tags:
    article_list.append(tag.text)

  article = " ".join([str(item) for item in article_list])
  articles.append(article)

In [ ]:
news_dict = {
    'headline' : headlines,
    'date_time' : date_time,
    'link' : news_urls,
    'news' : articles
}

news_df = pd.DataFrame(news_dict)

In [ ]:
news_df

In [ ]:
news_df.to_csv('/content/drive/MyDrive/Projects/news scraping/scraped_news.csv', index=None)